In [ ]:
from spade_bdi.bdi import BDIAgent
from spade.message import Message
import numpy as np

weights = np.array([
            [0.7,0.3],
            [0.4,0.6],
        ])

class user_moderator(BDIAgent):
    def __init__(self, jid, password):
        super().__init__(jid, password)


    async def python_consensus(self, msg_id, text_score, user_score):
        opinions = np.array([float(text_score), float(user_score)])
        consensus = french_degroot(weights, opinions)
        if consensus > 0.5:
            action = "keep"
        elif consensus > 0:
            action = "remove"
        else:
            action = "ban"
        await self.agent.bdi.set_belief("final_decision", msg_id, action, consensus)
        return True
    
    def train(weights, text_score, user_score, actual_result):
    # This function is used to train the weights based on the actual result
    
        opinions = np.array([float(text_score), float(user_score)])
        results = check_correctness(opinions, actual_result)
        adjust_weights(weights, results)

def french_degroot(weights, opinions):
    threshold = 0.01
    # protect the original opinions (optional)
    # opinions = opinions.copy()  

    for _ in range(100):
        new_opinions = np.dot(weights, opinions)
        if np.all(np.abs(new_opinions - opinions) < threshold):
            break
        opinions = new_opinions
    
    return opinions[0]

def check_correctness(opinions, actual_result):
    # This function checks if the agents' opinions match the actual result

    classification = []
    for i in range(len(opinions)):
        if opinions[i] > 0.5:
            classification.append("keep")
        elif opinions[i] > 0:
            classification.append("remove")
        else:
            classification.append("ban")

    results = [classification[i] == actual_result for i in range(len(classification))]
    return results

def adjust_weights(weights, results):
    # Adjust an agents influence based on results

    # If the results are the same, no change is needed
    if results[0] == results[1]:
        pass
    
    # If the results are different, adjust the weights
    # n = learning rate
    n = 0.1
    for item in weights:
        for i in range(len(item)):
            if results[i]:
                item[i] = round(item[i] + n * (1 - item[i]),2)
            else:
                item[i] = round(item[i] - n * item[i],2)
        item = item / np.sum(item)

def decide_action(msg_score, user_score):
    x = np.array([msg_score, user_score])
    consensus = french_degroot(x)
    if consensus > 0.5:
        return "keep"
    elif consensus > 0:
        return "remove"
    else:
        return "ban"
    
async def setup(self):
    print("Setting up user moderator agent...")
    self.set_internal_action("python_consensus", self.python_consensus)


